In [ ]:
#Bibliotecas
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

#Carregando dataset
df = pd.read_csv('Mikael Krug - Titanic - Mikael Krug - Titanic.csv')

#Parte 1
print(df.head())  # Mostra as 5 primeiras linhas do DataFrame
print(df.info())  # Mostra informações gerais: colunas, tipos de dados e valores nulos
print(df.describe())  # Mostra estatísticas descritivas das colunas numéricas
print(df['Sex'].value_counts())  # Conta quantas vezes cada valor aparece na coluna 'Sex'
print(df['Embarked'].value_counts())  # Conta quantos passageiros embarcaram em cada porto
print(df.isnull().sum())  # Mostra o total de valores nulos (faltando) em cada coluna

#Parte 2
df = df.drop_duplicates()

if 'Age' in df.columns:
    df['Age'] = df['Age'].fillna(df['Age'].median())

if 'Embarked' in df.columns and not df['Embarked'].mode().empty:
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

cols_to_drop = ['deck', 'embark_town', 'alive', 'who', 'adult_male', 'alone']
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

if 'Survived' in df.columns:
    df['Survived'] = df['Survived'].astype(bool)

print(df.isnull().sum())

#Parte 3
def categorize_age(age):
    if age < 18: return 'Criança'
    elif age < 60: return 'Adulto'
    else: return 'Idoso'

df['faixa_etaria'] = df['Age'].apply(categorize_age)

survivors = df[df['Survived'] == True]
print(survivors.head())

grouped = df.groupby(['pclass','sex']).agg({'Survived':'mean','Age':'mean','Fare':'median'}).reset_index()
print(grouped)

print(df.sort_values(by='Fare',ascending=False).head())

#Parte 4
sns.set(style='whitegrid')

plt.figure(figsize=(8,5))
sns.countplot(data=df, x='sex',hue='Survived',palette='Set2')
plt.xlabel('Sexo')
plt.ylabel('Contagem')
plt.show()

plt.figure(figsize=(8,5))
sns.boxplot(data=df,x='pclass',y='Age',hue='Survived',palette='Pastel1')
plt.title('Distribuição de Idade Classe e Sobrevivência')
plt.xlabel('Classe Social')
plt.ylabel('Idade')
plt.show()

numeric_df = df.select_dtypes
()